In [ ]:
# Retrieve image paths in given directories and save them to file

###########################
###### CONFIGURATION ######
img_dirs = ['~/Desktop/161_1025', '~/Desktop/163_1027'] # list of directories with images
img_exts = ['.jpg', '.JPG']       # valid image extensions
imagelist_file = 'imagelist.txt'  # file with image paths
###########################

import os

def img_path_gen():
    for dir in img_dirs:
        for item in os.listdir(os.path.expanduser(dir)):
            path2item = os.path.join(os.path.expanduser(dir), item)
            if os.path.isfile(path2item):
                if os.path.splitext(path2item)[-1] in img_exts:
                    yield path2item

with open(os.path.expanduser(imagelist_file), 'w') as file:                  
    for img_path in img_path_gen():
        print(img_path, file=file)

print('Image paths saved to', os.path.expanduser(imagelist_file))

In [ ]:
# YOLOv3 in openCV

# Based on
# https://github.com/arunponnusamy/object-detection-opencv 
# by Arun Ponnusamy (http://www.arunponnusamy.com)

###########################
###### CONFIGURATION ######

# jupyter matplotlib configs:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# YOLOv3 files from https://github.com/pjreddie/darknet
imagelist_file = 'imagelist.txt' # file with image paths
classes_file = '~/git/darknet/data/coco.names' # coco class labels
model_file = '~/git/darknet/cfg/yolov3.cfg'    # model definition
weights_file = '~/git/darknet/yolov3.weights'  # model weights (coco)

show_images = False # show images with detected objects

resolution = 960    # default reslution 416
###########################

import os
import json
import cv2
import numpy as np
from matplotlib import pyplot as plt

def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 10)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 3, color, 5)


imagelist_file = os.path.expanduser(imagelist_file)
classes_file = os.path.expanduser(classes_file)
model_file = os.path.expanduser(model_file)
weights_file = os.path.expanduser(weights_file)

detected_objects = {}
counter = 0
    
with open(imagelist_file, 'r') as source:
    for img_path in source.readlines():
        counter += 1
        print ('\r', counter, ': ', img_path.strip(), end='')
        detected_objects[img_path.strip()] = []
        
        image = cv2.imread(img_path.strip())

        Height, Width = image.shape[0:2]
        scale = 1/255

        classes = None

        with open(classes_file, 'r') as f:
            classes = [line.strip() for line in f.readlines()]

        COLORS = np.random.uniform(100, 230, size=(len(classes), 3))

        net = cv2.dnn.readNet(weights_file, model_file)
        blob = cv2.dnn.blobFromImage(image, scale, (resolution,resolution), (0,0,0), True, crop=False)

        net.setInput(blob)

        outs = net.forward(get_output_layers(net))

        class_ids = []
        confidences = []
        boxes = []
        conf_threshold = 0.6
        nms_threshold = 0.4


        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

        for i in cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold):
            i = i[0]
            box = boxes[i]
            x0, y0, w, h = [round(i) for i in box]
            x0 = max(0, x0)
            y0 = max(0, y0)
            x1 = min(Width, x0 + w)
            y1 = min(Height, y0 + h)
            
            #print('{:15} {:.2f}, {:4}, {:4}, {:4}, {:4}'.format(classes[class_ids[i]], confidences[i], x, y, w, h)) 
            
            detected_objects[img_path.strip()].append({'object':classes[class_ids[i]], 
                                                       'confidence':'%.2f' % confidences[i],
                                                       'x0': x0, 'y0': y0, 'x1': x1, 'y1': y1})
            if show_images: draw_prediction(image, class_ids[i], confidences[i], x, y, x+w, y+h)

        if show_images:
            plt.figure(figsize=(12,9))
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.show()
            
print('\nObject detection completed')

In [ ]:
# save car coordinates to a file

###########################
###### CONFIGURATION ######

detected_objs_file = 'detected_objects.txt'

###########################


with open(os.path.expanduser(detected_objs_file), 'w') as file:                  
    json.dump(detected_objects, file)
    print ('Detections saved to', detected_objs_file)